<a href="https://colab.research.google.com/github/IdjiotSandwiches/face-emotion-recognition/blob/data-augmentation/data_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import os
import cv2
import albumentations as A
from albumentations.core.composition import OneOf
from albumentations import HorizontalFlip, RandomBrightnessContrast, ShiftScaleRotate
from albumentations import Normalize
from tqdm import tqdm
import numpy as np

d:\PythonProject\env\Lib\site-packages\albumentations\__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.23 (you have 1.4.22). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [8]:
transform = A.Compose([
    A.Resize(224, 224),
    A.RandomBrightnessContrast(p=0.5),
    A.HorizontalFlip(p=0.5),
    A.RandomScale(scale_limit=0.2, p=0.5),
    A.Rotate(limit=15, p=0.5),
])

In [9]:
def augment_and_save(input_dir, output_dir, num_augments=3):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for class_name in os.listdir(input_dir):
        if(class_name == 'disgust'):
            continue
        class_path = os.path.join(input_dir, class_name)
        if not os.path.isdir(class_path):
            continue 

        output_class_path = os.path.join(output_dir, class_name)
        os.makedirs(output_class_path, exist_ok=True)

        for img_name in tqdm(os.listdir(class_path), desc=f"Processing {class_name}"):
            img_path = os.path.join(class_path, img_name)
            image = cv2.imread(img_path)
            if image is None:
                continue
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            original_output_path = os.path.join(output_class_path, img_name)
            cv2.imwrite(original_output_path, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))

            for i in range(num_augments):
                augmented = transform(image=image)['image']
                aug_img_name = f"{os.path.splitext(img_name)[0]}_aug_{i}.jpg"
                aug_img_path = os.path.join(output_class_path, aug_img_name)
                cv2.imwrite(aug_img_path, cv2.cvtColor(augmented, cv2.COLOR_RGB2BGR))

In [10]:
input_dir = "C:\\Users\\vinar\\Downloads\\FER2013-down-sampling"
output_dir = "C:\\Users\\vinar\\Downloads\\FER2013-augmented"
augment_and_save(input_dir, output_dir, num_augments=3)

Processing surprise: 100%|██████████| 3000/3000 [00:19<00:00, 150.35it/s]
